In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys

if '..' not in sys.path:
    sys.path.append('..')

from data import ThreeBallsCollisionDataset

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn.functional import mse_loss as mse_pytorch

import reservoirpy as rpy
from reservoirpy.observables import mse as mse_esn

from itertools import product
from tqdm.notebook import tqdm_notebook

from models import VanilaRNN, GRU, LSTM, ESN, SeqESN, ParallelESN, GroupedESN

rpy.verbosity(0)

0

# Importing data

In [2]:
batch_size = 32

train_data, test_data = ThreeBallsCollisionDataset.train_test_split('../data/raw/three-balls-collisions-limited-free-fall', test_frac=0.2, max_actions=10000, free_fall_fraction=0.15)

train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size)

252349it [20:05, 209.26it/s]


KeyboardInterrupt: 

In [3]:
dir(ThreeBallsCollisionDataset)

['ClassicalMechanicsDataset',
 'ThreeBallsCollisionsDataset',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'np',
 'os',
 'phyre',
 'product',
 're',
 'torch',
 'tqdm']